In [1]:
import logging
import os
from typing import Tuple

api_key = os.environ["PALM_API_KEY"]
logging.basicConfig()
logger = logging.getLogger()
logger.setLevel(logging.WARNING)

In [2]:
import semantic_kernel as sk
import semantic_kernel.connectors.ai.google_palm as sk_gp
import semantic_kernel.connectors.ai.hugging_face as sk_hf

kernel = sk.Kernel(log=logger)
kernel.add_text_completion_service(
    "models/text-bison-001",
    sk_gp.GooglePalmTextCompletion("models/text-bison-001", api_key),
)
# NOTE: Inaccurate results when using this.
# kernel.add_text_embedding_generation_service(
#     "models/embedding-gecko-001",
#     sk_gp.GooglePalmTextEmbedding("models/embedding-gecko-001", api_key),
# )
kernel.add_text_embedding_generation_service(
    "BAAI/bge-small-en-v1.5", sk_hf.HuggingFaceTextEmbedding("BAAI/bge-small-en-v1.5")
)
# kernel.add_text_embedding_generation_service(
#     "sentence-transformers/all-MiniLM-L6-v2",
#     sk_hf.HuggingFaceTextEmbedding("sentence-transformers/all-MiniLM-L6-v2"),
# )
kernel.register_memory_store(memory_store=sk.memory.VolatileMemoryStore())
kernel.import_skill(sk.core_skills.TextMemorySkill())

/Users/alextanhongpin/Documents/python/python-palm-semantic-kernel/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'recall': <semantic_kernel.orchestration.sk_function.SKFunction at 0x123ed3c90>,
 'save': <semantic_kernel.orchestration.sk_function.SKFunction at 0x123ed3390>}

In [3]:
async def populate_memory(kernel: sk.Kernel) -> None:
    # Add some documents to the semantic memory
    await kernel.memory.save_information_async(
        "aboutMe", id="info1", text="My name is Andrea"
    )
    await kernel.memory.save_information_async(
        "aboutMe", id="info2", text="I currently work as a tour guide"
    )
    await kernel.memory.save_information_async(
        "aboutMe", id="info3", text="I've been living in Seattle since 2005"
    )
    await kernel.memory.save_information_async(
        "aboutMe", id="info4", text="I visited France and Italy five times since 2015"
    )
    await kernel.memory.save_information_async(
        "aboutMe", id="info5", text="My family is from New York"
    )

In [4]:
await populate_memory(kernel)

In [5]:
async def search_memory_examples(kernel: sk.Kernel) -> None:
    questions = [
        "what's my name",
        "where do I live?",
        "where's my family from?",
        "where have I traveled?",
        "what do I do for work",
    ]
    for question in questions:
        print(f"Question: {question}")
        result = await kernel.memory.search_async("aboutMe", question)
        print(f"Answer: {result[0].text}\n")

In [6]:
await search_memory_examples(kernel)

Question: what's my name
Answer: My name is Andrea

Question: where do I live?
Answer: I've been living in Seattle since 2005

Question: where's my family from?
Answer: My family is from New York

Question: where have I traveled?
Answer: I currently work as a tour guide

Question: what do I do for work
Answer: I currently work as a tour guide



In [7]:
async def setup_chat_with_memory(
    kernel: sk.Kernel,
) -> Tuple[sk.SKFunctionBase, sk.SKContext]:
    sk_prompt = """
    ChatBot can have a conversation with you about any topic.
    It can give explicit instructions or say 'I don't know' if
    it does not have an answer.
    
    Information about me, from previous conversations:
    - {{$fact1}} {{recall $fact1}}
    - {{$fact2}} {{recall $fact2}}
    - {{$fact3}} {{recall $fact3}}
    - {{$fact4}} {{recall $fact4}}
    - {{$fact5}} {{recall $fact5}}
    
    Chat:
    {{$chat_history}}
    User: {{$user_input}}
    ChatBot: """.strip()

    chat_func = kernel.create_semantic_function(
        sk_prompt, max_tokens=200, temperature=0.8
    )

    context = kernel.create_new_context()
    context["fact1"] = "what is my name?"
    context["fact2"] = "where do I live?"
    context["fact3"] = "where's my family from?"
    context["fact4"] = "where have I traveled?"
    context["fact5"] = "what do I do for work?"

    context[sk.core_skills.TextMemorySkill.COLLECTION_PARAM] = "aboutMe"
    # The RelevanceParam is used in memory search and is a measure of the relevance score
    # from 0.0 to 1.0, where 1.0 means a perfect match.
    context[sk.core_skills.TextMemorySkill.RELEVANCE_PARAM] = 0.8

    context["chat_history"] = ""

    return chat_func, context

In [8]:
async def chat(
    kernel: sk.Kernel, chat_func: sk.SKFunctionBase, context: sk.SKContext
) -> bool:
    try:
        user_input = input("User:> ")
        context["user_input"] = user_input
        print(f"User:> {user_input}")
    except KeyboardInterrupt:
        print("\n\nExiting chat...")
        return False
    except EOFError:
        print("\n\nExiting chat...")
        return False

    if user_input == "exit":
        print("\n\nExiting chat...")
        return False

    answer = await kernel.run_async(chat_func, input_vars=context.variables)
    context["chat_history"] += f"\nUser:> {user_input}\nChatBot:> {answer}\n"

    print(f"ChatBot:> {answer}")
    return True

In [9]:
print("Populating memory...")
await populate_memory(kernel)

print("Asking questions... (manually)")
await search_memory_examples(kernel)

print("Setting up a chat (with memory!)")
chat_func, context = await setup_chat_with_memory(kernel)

print("Begin chatting (type 'exit' to exit):\n")
chatting = True
while chatting:
    chatting = await chat(kernel, chat_func, context)

Populating memory...
Asking questions... (manually)
Question: what's my name
Answer: My name is Andrea

Question: where do I live?
Answer: I've been living in Seattle since 2005

Question: where's my family from?
Answer: My family is from New York

Question: where have I traveled?
Answer: I currently work as a tour guide

Question: what do I do for work
Answer: I currently work as a tour guide

Setting up a chat (with memory!)
Begin chatting (type 'exit' to exit):



User:>  name the places I travelled to


User:> name the places I travelled to
ChatBot:> You have traveled to New York, Las Vegas, and Paris.


User:>  wrong


User:> wrong
ChatBot:> > I don't know.


User:>  exit


User:> exit


Exiting chat...
